# Ingeniería de Características (Feature Engineering)
## Proyecto de Mantenimiento Predictivo - Moto-Compresores

Este notebook contiene la creación y selección de características para el modelo predictivo.

### Objetivos:
- Crear características derivadas
- Generar indicadores de tendencia
- Seleccionar características relevantes
- Crear variables de degradación

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.signal import detrend
from scipy.fft import fft, fftfreq
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression
import warnings
import gc

# Configuración
warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("✅ Librerías importadas correctamente")
print(f"📅 Notebook ejecutado: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 1. Carga de Datos Procesados

In [ ]:
# Configurar rutas
data_processed_path = Path("data/processed")
eventos_path = Path("eventos")
output_path = Path("data/processed")

# Verificar archivos disponibles
print("🔍 Buscando archivos de datos preprocesados...")

parquet_files = list(data_processed_path.glob("*.parquet"))
csv_files = list(data_processed_path.glob("*.csv"))

print(f"📦 Archivos Parquet encontrados: {len(parquet_files)}")
for file in parquet_files:
    print(f"   • {file.name}")
    
print(f"📄 Archivos CSV encontrados: {len(csv_files)}")
for file in csv_files:
    print(f"   • {file.name}")

# Cargar el dataset principal
try:
    if parquet_files and 'timeseries_data.parquet' in [f.name for f in parquet_files]:
        print("\n💾 Cargando timeseries_data.parquet...")
        df = pd.read_parquet(data_processed_path / "timeseries_data.parquet")
        print("✅ Parquet cargado exitosamente")
    elif csv_files and 'clean_timeseries_data.csv' in [f.name for f in csv_files]:
        print("\n💾 Cargando clean_timeseries_data.csv...")
        df = pd.read_csv(data_processed_path / "clean_timeseries_data.csv", 
                        index_col=0, parse_dates=True)
        print("✅ CSV cargado exitosamente")
    else:
        print("❌ No se encontró archivo de datos procesados")
        print("   Buscando archivo más reciente...")
        all_files = parquet_files + csv_files
        if all_files:
            latest_file = max(all_files, key=lambda x: x.stat().st_mtime)
            print(f"   Cargando: {latest_file.name}")
            if latest_file.suffix == '.parquet':
                df = pd.read_parquet(latest_file)
            else:
                df = pd.read_csv(latest_file, index_col=0, parse_dates=True)
            print("✅ Archivo cargado exitosamente")
        else:
            raise FileNotFoundError("No se encontraron archivos de datos")
            
except Exception as e:
    print(f"❌ Error al cargar datos: {str(e)}")
    raise

# Información del dataset cargado
print(f"\n📊 Dataset cargado:")
print(f"   📐 Dimensiones: {df.shape[0]:,} filas × {df.shape[1]} columnas")
print(f"   📅 Período: {df.index.min()} → {df.index.max()}")
print(f"   ⏱️  Frecuencia: {pd.infer_freq(df.index) if hasattr(df.index, 'freq') else 'No detectada'}")
print(f"   💾 Memoria: {df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

# Preview de los datos
print("\n👀 Primeras 3 filas:")
print(df.head(3))

print("\n📋 Tipos de datos:")
print(df.dtypes.value_counts())

## 2. Selección Inteligente de Variables

Algoritmo automático para seleccionar las variables más relevantes para el mantenimiento predictivo.

In [ ]:
def seleccionar_variables_relevantes(df, min_completitud=0.7, min_variabilidad=0.01, 
                                   max_variables=50):
    """
    Selecciona automáticamente las variables más relevantes para mantenimiento predictivo
    """
    print("🎯 Ejecutando selección inteligente de variables...")
    
    # Solo variables numéricas
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    print(f"   📊 Variables numéricas disponibles: {len(numeric_cols)}")
    
    scores = []
    
    for col in numeric_cols:
        try:
            serie = df[col].dropna()
            
            if len(serie) == 0:
                continue
                
            # 1. Completitud de datos (peso: 0.3)
            completitud = len(serie) / len(df)
            score_completitud = completitud if completitud >= min_completitud else 0
            
            # 2. Variabilidad (peso: 0.25)
            cv = serie.std() / serie.mean() if serie.mean() != 0 else 0
            score_variabilidad = min(cv, 2.0) / 2.0 if cv >= min_variabilidad else 0
            
            # 3. Relevancia física para moto-compresores (peso: 0.45)
            relevancia_fisica = 0.0
            col_lower = col.lower()
            
            # Variables críticas (relevancia alta)
            if any(term in col_lower for term in ['presion', 'pressure', 'temp', 'temperatura', 'vibr']):
                relevancia_fisica = 1.0
            # Variables importantes (relevancia media-alta)  
            elif any(term in col_lower for term in ['motor', 'corriente', 'current', 'voltage', 'voltaje', 'power', 'potencia']):
                relevancia_fisica = 0.8
            # Variables útiles (relevancia media)
            elif any(term in col_lower for term in ['flujo', 'flow', 'nivel', 'level', 'rpm', 'velocidad', 'speed']):
                relevancia_fisica = 0.6
            # Variables auxiliares (relevancia baja)
            else:
                relevancia_fisica = 0.3
            
            # Score compuesto
            score_final = (0.3 * score_completitud + 
                          0.25 * score_variabilidad + 
                          0.45 * relevancia_fisica)
            
            scores.append({
                'variable': col,
                'score_final': score_final,
                'completitud': completitud,
                'variabilidad': cv,
                'relevancia_fisica': relevancia_fisica,
                'media': serie.mean(),
                'std': serie.std()
            })
            
        except Exception as e:
            print(f"      ⚠️  Error procesando {col}: {str(e)}")
            continue
    
    # Convertir a DataFrame y ordenar por score
    df_scores = pd.DataFrame(scores)
    df_scores = df_scores.sort_values('score_final', ascending=False)
    
    # Seleccionar mejores variables
    variables_seleccionadas = df_scores.head(max_variables)['variable'].tolist()
    
    print(f"✅ Variables seleccionadas: {len(variables_seleccionadas)}")
    print("\n🏆 Top 10 variables por relevancia:")
    for i, row in variables_seleccionadas[:10].iterrows() if hasattr(variables_seleccionadas[:10], 'iterrows') else enumerate(df_scores.head(10).itertuples()):
        if hasattr(variables_seleccionadas[:10], 'iterrows'):
            print(f"   {i+1:2d}. {row['variable']} (score: {row['score_final']:.3f})")
        else:
            print(f"   {i+1:2d}. {row.variable} (score: {row.score_final:.3f})")
    
    return variables_seleccionadas, df_scores

# Ejecutar selección
variables_seleccionadas, df_scores = seleccionar_variables_relevantes(df)

# Crear dataset filtrado
df_selected = df[variables_seleccionadas].copy()
print(f"\n📐 Dataset filtrado: {df_selected.shape[0]:,} × {df_selected.shape[1]}")
print(f"💾 Reducción de memoria: {(df.memory_usage(deep=True).sum() - df_selected.memory_usage(deep=True).sum()) / 1024**2:.1f} MB")

## 3. Características de Ventanas Móviles (Rolling Features)

Creación de estadísticas agregadas en diferentes ventanas temporales para capturar tendencias y patrones de degradación.

In [ ]:
def crear_rolling_features(df, ventanas=['6H', '24H', '72H'], 
                          estadisticas=['mean', 'std', 'min', 'max'], 
                          variables_prioritarias=None):
    """
    Crea características de ventanas móviles para múltiples estadísticas
    """
    print("📈 Creando características de ventanas móviles...")
    
    if variables_prioritarias is None:
        variables_prioritarias = df.columns[:20]  # Primeras 20 por defecto
    
    print(f"   📊 Variables a procesar: {len(variables_prioritarias)}")
    print(f"   ⏱️  Ventanas temporales: {ventanas}")
    print(f"   📋 Estadísticas: {estadisticas}")
    
    rolling_features = pd.DataFrame(index=df.index)
    contador_features = 0
    
    for ventana in ventanas:
        print(f"\n   🔄 Procesando ventana {ventana}...")
        
        for variable in variables_prioritarias:
            try:
                # Crear rolling window
                rolling = df[variable].rolling(window=ventana, min_periods=1)
                
                for stat in estadisticas:
                    nombre_feature = f"{variable}_roll_{ventana}_{stat}"
                    
                    if stat == 'mean':
                        rolling_features[nombre_feature] = rolling.mean()
                    elif stat == 'std':
                        rolling_features[nombre_feature] = rolling.std()
                    elif stat == 'min':
                        rolling_features[nombre_feature] = rolling.min()
                    elif stat == 'max':
                        rolling_features[nombre_feature] = rolling.max()
                    elif stat == 'median':
                        rolling_features[nombre_feature] = rolling.median()
                    elif stat == 'skew':
                        rolling_features[nombre_feature] = rolling.skew()
                    elif stat == 'kurt':
                        rolling_features[nombre_feature] = rolling.kurt()
                    
                    contador_features += 1
                    
            except Exception as e:
                print(f"      ⚠️  Error con {variable}: {str(e)}")
                continue
    
    # Crear características adicionales de volatilidad y tendencia
    print(f"\n   📊 Creando características de volatilidad y tendencia...")
    
    for ventana in ['24H', '72H']:  # Solo ventanas más grandes
        for variable in variables_prioritarias[:10]:  # Solo top variables
            try:
                rolling = df[variable].rolling(window=ventana, min_periods=1)
                
                # Coeficiente de variación
                mean_val = rolling.mean()
                std_val = rolling.std()
                cv_name = f"{variable}_roll_{ventana}_cv"
                rolling_features[cv_name] = std_val / (mean_val + 1e-6)  # Evitar división por 0
                
                # Tendencia (pendiente)
                trend_name = f"{variable}_roll_{ventana}_trend"
                rolling_features[trend_name] = rolling.apply(
                    lambda x: np.polyfit(range(len(x)), x, 1)[0] if len(x) > 1 else 0
                )
                
                contador_features += 2
                
            except Exception as e:
                continue
    
    print(f"✅ Rolling features creadas: {contador_features}")
    print(f"📐 Dimensiones: {rolling_features.shape[0]:,} × {rolling_features.shape[1]}")
    
    return rolling_features

# Ejecutar creación de rolling features
rolling_features = crear_rolling_features(df_selected, 
                                        ventanas=['6H', '24H', '72H'],
                                        variables_prioritarias=df_selected.columns[:15])

print(f"\n💾 Memoria rolling features: {rolling_features.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

## 4. Características de Lag (Retrasos Temporales)

Creación de variables que capturan el comportamiento histórico del equipo para predecir comportamiento futuro.

In [ ]:
def crear_lag_features(df, lags=['2H', '12H', '48H'], 
                      variables_prioritarias=None, max_features=150):
    """
    Crea características de lag (retrasos temporales)
    """
    print("⏪ Creando características de lag (retrasos temporales)...")
    
    if variables_prioritarias is None:
        variables_prioritarias = df.columns[:10]  # Primeras 10 por defecto
    
    print(f"   📊 Variables a procesar: {len(variables_prioritarias)}")
    print(f"   ⏱️  Lags temporales: {lags}")
    
    lag_features = pd.DataFrame(index=df.index)
    contador_features = 0
    
    for lag in lags:
        print(f"\n   🔄 Procesando lag {lag}...")
        
        for variable in variables_prioritarias:
            if contador_features >= max_features:
                print(f"   ⚠️  Límite de {max_features} features alcanzado")
                break
                
            try:
                # Convertir lag a frecuencia pandas
                lag_periods = pd.Timedelta(lag)
                
                # Crear feature de lag
                nombre_feature = f"{variable}_lag_{lag}"
                lag_features[nombre_feature] = df[variable].shift(freq=lag_periods)
                
                contador_features += 1
                
            except Exception as e:
                print(f"      ⚠️  Error con {variable} lag {lag}: {str(e)}")
                continue
        
        if contador_features >= max_features:
            break
    
    # Crear características de diferencias (cambios entre períodos)
    print(f"\n   📊 Creando características de diferencias...")
    
    for variable in variables_prioritarias[:8]:  # Solo top variables
        if contador_features >= max_features:
            break
            
        try:
            # Diferencia de 12H
            diff_name = f"{variable}_diff_12H"
            lag_features[diff_name] = df[variable] - df[variable].shift(freq='12H')
            
            # Cambio porcentual de 24H
            pct_name = f"{variable}_pct_change_24H"
            lag_features[pct_name] = df[variable].pct_change(
                periods=pd.Timedelta('24H') // pd.infer_freq(df.index) if pd.infer_freq(df.index) else 1
            )
            
            contador_features += 2
            
        except Exception as e:
            continue
    
    print(f"✅ Lag features creadas: {contador_features}")
    print(f"📐 Dimensiones: {lag_features.shape[0]:,} × {lag_features.shape[1]}")
    
    return lag_features

# Ejecutar creación de lag features
lag_features = crear_lag_features(df_selected, 
                                lags=['2H', '12H', '48H'],
                                variables_prioritarias=df_selected.columns[:12])

print(f"\n💾 Memoria lag features: {lag_features.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

## 5. Dataset Final de Características y Guardado

In [ ]:
# Importar función de guardado completo
exec(open('guardar_completo.py').read())

# Consolidar todas las características
print("🔗 Consolidando dataset final de características...")

# Crear dataset base con variables originales seleccionadas
dataset_final = df_selected.copy()
print(f"📊 Variables originales: {dataset_final.shape[1]}")

# Agregar rolling features (seleccionar las más importantes para evitar overfitting)
if not rolling_features.empty:
    # Filtrar rolling features con variabilidad significativa
    rolling_features_filtered = rolling_features.dropna(axis=1, thresh=int(0.7 * len(rolling_features)))
    rolling_features_filtered = rolling_features_filtered.select_dtypes(include=[np.number])
    
    # Seleccionar solo features con variación suficiente
    features_con_variacion = []
    for col in rolling_features_filtered.columns:
        if rolling_features_filtered[col].std() > 1e-6:  # Evitar features constantes
            features_con_variacion.append(col)
    
    rolling_features_final = rolling_features_filtered[features_con_variacion[:100]]  # Top 100
    dataset_final = pd.concat([dataset_final, rolling_features_final], axis=1)
    print(f"📈 Rolling features agregadas: {rolling_features_final.shape[1]}")

# Agregar lag features
if not lag_features.empty:
    # Filtrar lag features con variabilidad significativa
    lag_features_filtered = lag_features.dropna(axis=1, thresh=int(0.5 * len(lag_features)))
    lag_features_filtered = lag_features_filtered.select_dtypes(include=[np.number])
    
    # Seleccionar solo features con variación suficiente
    features_con_variacion = []
    for col in lag_features_filtered.columns:
        if lag_features_filtered[col].std() > 1e-6:
            features_con_variacion.append(col)
    
    lag_features_final = lag_features_filtered[features_con_variacion[:50]]  # Top 50
    dataset_final = pd.concat([dataset_final, lag_features_final], axis=1)
    print(f"⏪ Lag features agregadas: {lag_features_final.shape[1]}")

# Limpieza final del dataset
print("\n🧹 Limpieza final del dataset...")

# Eliminar columnas completamente nulas o constantes
dataset_final = dataset_final.dropna(axis=1, how='all')
for col in dataset_final.select_dtypes(include=[np.number]).columns:
    if dataset_final[col].std() < 1e-10:  # Prácticamente constante
        dataset_final = dataset_final.drop(col, axis=1)

# Convertir a float32 para optimizar memoria
numeric_columns = dataset_final.select_dtypes(include=[np.number]).columns
dataset_final[numeric_columns] = dataset_final[numeric_columns].astype(np.float32)

# Información final
print(f"\n📊 Dataset final consolidado:")
print(f"   📐 Dimensiones: {dataset_final.shape[0]:,} filas × {dataset_final.shape[1]} columnas")
print(f"   📅 Período: {dataset_final.index.min()} → {dataset_final.index.max()}")
print(f"   💾 Memoria: {dataset_final.memory_usage(deep=True).sum() / 1024**2:.1f} MB")
print(f"   🔢 Variables numéricas: {len(dataset_final.select_dtypes(include=[np.number]).columns)}")

# Mostrar composición del dataset
print(f"\n📋 Composición del dataset final:")
print(f"   🔵 Variables originales: {len(df_selected.columns)}")
if not rolling_features.empty:
    print(f"   📈 Rolling features: {rolling_features_final.shape[1] if 'rolling_features_final' in locals() else 0}")
if not lag_features.empty:
    print(f"   ⏪ Lag features: {lag_features_final.shape[1] if 'lag_features_final' in locals() else 0}")

# Guardar dataset usando función completa
print(f"\n💾 Guardando dataset final de características...")

try:
    resultados_guardado = guardar_dataset_final(
        df=dataset_final,
        ruta_destino=output_path,
        nombre_base='features_dataset',
        archivos_procesados=['datos_procesados_exitosamente'],
        archivos_fallidos=[],
        valores_interpolados=0,
        valores_clipped=0
    )
    
    print(f"\n✅ FEATURE ENGINEERING COMPLETADO")
    print(f"📁 Archivos generados en data/processed/:")
    for formato, resultado in resultados_guardado.items():
        if resultado['exito']:
            print(f"   ✅ features_dataset.{formato} ({resultado['tamaño_mb']:.1f} MB)")
    
    print(f"\n🎯 Dataset listo para entrenamiento de modelos")
    print(f"➡️  Siguiente fase: 04_model_training.ipynb")
    
except Exception as e:
    print(f"❌ Error al guardar: {str(e)}")
    # Guardar al menos en CSV como respaldo
    backup_file = output_path / 'features_dataset_backup.csv'
    dataset_final.to_csv(backup_file)
    print(f"💾 Guardado respaldo en: {backup_file}")

# Liberar memoria
del df, df_selected, rolling_features, lag_features
gc.collect()

print(f"\n🏁 Feature Engineering finalizado exitosamente")
print(f"📊 Total de características creadas: {dataset_final.shape[1]}")
print(f"🔗 Dataset listo para la fase de modelado")